In [3]:
# !pip install -U scikit-learn

     |████████████████████████████████| 22.3 MB 187 kB/s eta 0:00:01    |██████████▋                     | 7.4 MB 5.0 MB/s eta 0:00:03     |██████████████▉                 | 10.3 MB 7.9 MB/s eta 0:00:02
     |████████████████████████████████| 303 kB 4.9 MB/s eta 0:00:01


In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss
from sklearn.metrics import make_scorer

# TP2 - Random Forest Model

El objetivo es crear un primer modelo básico de random forest, sin feature engineering ni hyperparameters tunning, para poder tener un primer resultado local. Luego, realizar un primer submit en kaggle y en base a eso ir mejorando el modelo 

In [25]:
train = pd.read_csv("../datasets/first-rf-model-train.csv")
test = pd.read_csv("../datasets/first-rf-model-test.csv")

train['Month'] = pd.to_datetime(train['Month']).dt.month
test['Month'] = pd.to_datetime(test['Month']).dt.month

In [26]:
test.head(2)

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Opportunity_ID,Sales_Contract_No,ASP,ASP_(converted),Month,Delivery_Year,...,Last_Modified_By_2,Last_Modified_By_3,Last_Modified_By_4,Last_Modified_By_5,Total_Taxable_Amount_Currency_0,Total_Taxable_Amount_Currency_1,Total_Taxable_Amount_Currency_2,Total_Amount_Currency_0,Total_Amount_Currency_1,Total_Amount_Currency_2
0,1,1,1,1,10689,6436,0.325,0.36761,5,2019,...,0,1,0,0,0,0,1,0,0,1
1,1,1,1,1,10689,6436,0.320,0.36195,5,2019,...,0,1,0,0,0,0,1,0,0,1


In [28]:
target = "target"
features = list(train.columns)
features.remove(target)
len(features)

110

In [29]:
X_train, y_train = train.loc[:,features],train.loc[:,target]
X_test = test

In [30]:
print(X_train.shape)
print(X_test.shape)

(16947, 110)
(2551, 110)


In [33]:
num_of_trees = 25
max_depth = 6
max_features = 10
min_to_split = 15
min_to_leaf = 1

rf_model = RandomForestClassifier(n_estimators=num_of_trees,\
                                     max_depth = max_depth,\
                                     max_features = max_features,\
                                     min_samples_split = min_to_split,\
                                     min_samples_leaf = min_to_leaf,\
                                     random_state=1
                                 )


run = cross_val_score(rf_model,X_train,y_train,cv=1+int(len(X_train)/15),scoring= make_scorer(log_loss))
run
#rf_model.fit(X_train,y_train)

array([1.15129255e+01, 1.84206807e+01, 4.60517019e+00, ...,
       9.99200722e-16, 7.40122348e+00, 2.46705546e+00])

In [41]:
run_df = pd.Series(run)
run_df.describe()

count    1.130000e+03
mean     1.575741e+00
std      3.221097e+00
min      9.992007e-16
25%      9.992007e-16
50%      9.992007e-16
75%      2.302638e+00
max      1.842073e+01
dtype: float64

In [45]:
rf_model.fit(X_train,y_train)
pred = rf_model.predict(X_test)

In [67]:
pred_df = pd.DataFrame(X_test["Opportunity_ID"]) 
pred_df["Target"] = pred
pred_df = pred_df.drop_duplicates("Opportunity_ID")
pred_df

,Opportunity_ID,Target
0,10689,1
3,10690,1
8,10691,1
9,10692,1
15,10693,1
...,...,...
2545,12364,1
2547,12365,1
2548,12366,1
2549,12367,0


In [70]:
pred_df.to_csv("../submits/rf_model1.csv",index=False)